In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [4]:
df = df['text']

In [5]:
df = pd.DataFrame(df)
df['id'] = df.index

In [6]:
from transformers import AutoTokenizer
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  


def chunk_text(text, chunk_size=300, overlap=50, max_length=1024):
    tokens = tokenizer.encode(text, max_length=max_length, truncation=False)  
    chunks = []

    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        if end == len(tokens):
            break
        start += chunk_size - overlap

    return chunks

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
df['chunks'] = df['text'].apply(lambda x: chunk_text(x))

chunk_rows = []
for _, row in df.iterrows():
    for i, chunk in enumerate(row['chunks']):
        chunk_rows.append({
            'id': row['id'],
            'chunk_id': i,
            'chunk_text': chunk
        })

chunk_df = pd.DataFrame(chunk_rows)

In [8]:
from sentence_transformers import SentenceTransformer


embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

chunk_texts = chunk_df['chunk_text'].tolist()
embeddings = embedder.encode(chunk_texts, show_progress_bar=True)

2025-06-25 14:36:46.740535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750862206.932341      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750862206.984567      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1540 [00:00<?, ?it/s]

In [9]:
!pip install faiss-cpu -q
import faiss
embedding_dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
embedding_array = np.array(embeddings).astype("float32")
index.add(embedding_array)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.6 MB/s eta 0:00:00:00:0100:01


In [10]:
def search_similar_chunks(query, top_k=5):

    query_embedding = embedder.encode([query]).astype("float32")
    

    distances, indices = index.search(query_embedding, top_k)
    

    matched_chunks = []
    for i in indices[0]:
        matched_chunks.append(chunk_df.iloc[i]['chunk_text'])
        
    return matched_chunks

In [11]:
query = "Presidential foreign policy announcements"
results = search_similar_chunks(query, top_k=3)

for idx, chunk in enumerate(results):
    print(f"Chunk {idx+1}:\n{chunk}\n{'-'*40}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chunk 1:
 bad thing” for the United States if Japan had nuclear weapons, and has said the same about South Korea and Saudi Arabia. Burt said the main theme he talked about to Trump was the need for the United States to pursue a “more realist foreign policy,” in which the United States would avoid seeking “regime change” abroad, and instead make protecting the United States and its interests the main policy goal. Trump, who became the Republican’s presumptive nominee last month after the last of his challengers dropped from the party’s race, articulated this theme again in a campaign speech late on Tuesday, as he claimed victory in the last Republican nominating contests. Under Reagan, Burt headed the State Department’s Bureau of Political-Military Affairs and then its European and Canadian Affairs bureau and served as U.S. ambassador to Germany from 1985 to 1989. Some of Burt’s contemporaries from the administrations of Presidents Reagan, George H.W. Bush and George W. Bush, have publi

In [12]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="microsoft/phi-2",device_map="auto")


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
!pip install gradio -q
import gradio as gr
import time

def rag_pipeline(query, prompt_template):
    import time
    start_time = time.time()

    chunks = search_similar_chunks(query, top_k=3)
    context = "\n\n".join(chunks)


    prompt = prompt_template.format(query=query, context=context).strip()

    outputs = pipe(
        prompt,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.8,
        repetition_penalty=1.1,
        return_full_text=False, 
        top_p=0.9
    )

    end_time = time.time()
    duration = end_time - start_time

    result = outputs[0]['generated_text'].strip()
    return f"{result}\n\n🕒 Inference time: {duration:.2f} seconds"



iface = gr.Interface(
    fn=rag_pipeline,
    inputs=[
        gr.Textbox(label="Query", placeholder="e.g., What were the main drivers behind Bitcoin's price surge in 2017?"),
        gr.Textbox(label="Prompt Template", lines=6, placeholder="e.g., You are a financial analyst...\n\nQuestion: {query}\n\nContext:\n{context}\n\nAnswer:")
    ],
    outputs="text",
    title="Custom Prompt RAG-powered News Q&A",
    description="Enter your own prompt and query. The model will answer using top-k relevant news chunks."
)

iface.launch(share=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://7350eb2a95377126f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# Notes on RAG System Experience and Prompt-Query Consistency

Initially, I tried to build a RAG system using both fake and true labeled news data. However, due to the influence of fake news, the model produced unusual and often nonsensical outputs. Therefore, I decided to exclude fake news and train the system only with true news data.

This also highlighted that:

- The relationship between the prompt and the query must be clear and consistent.  
- The **prompt template must include placeholders exactly as `{query}` and `{context}`**, enclosed in curly braces, because these are replaced programmatically with the actual user question and relevant context chunks. Changing or removing these placeholders breaks this dynamic insertion, causing irrelevant or meaningless answers.  
- Similarly, the prompt should end with an **explicit “Answer:”** to signal the model where to start generating the answer based strictly on the provided context and query. Omitting or altering this can confuse the model and degrade answer quality.

Otherwise, for example, when asking “What is Covid-19?”, the model might respond with meaningless answers like “Answer your question”.

Additionally, the inference time is excessively long, which poses a significant problem for practical use.

---

In summary, **data quality, prompt-query alignment (including exact placeholder usage), and inference speed are all critical factors for the performance of a RAG system.**  
Make sure your prompt template is written exactly like this in your UI or code:


